# Лабораторная работа №5 — Градиентный бустинг (Gradient Boosting)

Цель: построить бейзлайны для **классификации** и **регрессии** с использованием градиентного бустинга из `sklearn`, выбрать метрики качества и обосновать их выбор, затем оценить качество моделей на тестовой выборке.

Датасеты:
- **Классификация:** `customer_support_data.csv` — уровень удовлетворённости клиента (**используем 20% выборку от всего датасета**).
- **Регрессия:** `cwurData.csv` — показатель, связанный с мировым рейтингом университета.


In [84]:
import numpy as np
import pandas as pd

# Базовые импорты и настройки отображения
pd.set_option("display.max_columns", 200)


## 1) Метрики качества и обоснование

### Классификация (GradientBoostingClassifier)
- **Accuracy** — доля верных ответов; подходит как базовая метрика для сравнения моделей.
- **F1-score (macro)** — оценивает качество по классам равномерно и полезна при дисбалансе классов.

### Регрессия (GradientBoostingRegressor)
- **MAE** — средняя абсолютная ошибка, интерпретируется в единицах целевой переменной.
- **RMSE** — сильнее штрафует большие ошибки, полезна для оценки “крупных промахов”.
- **R²** — показывает долю объяснённой дисперсии (качество аппроксимации).


In [85]:
from sklearn.model_selection import train_test_split

# Зафиксируем random_state для воспроизводимости результатов
RANDOM_STATE = 42


## 2) Классификация: customer_support_data.csv (20% выборка) — бейзлайн на GradientBoostingClassifier


In [86]:
df_cls = pd.read_csv("customer_support_data.csv")

# Загрузили полный датасет; далее берём 20% согласно условию
df_cls.shape


(85907, 20)

In [87]:
# Берём случайную подвыборку 20% от всего датасета
df_cls_sample = df_cls.sample(frac=0.2, random_state=RANDOM_STATE).reset_index(drop=True)
df_cls_sample.shape


(17181, 20)

In [88]:
# Эвристика выбора target: по ключевым словам или по малому числу уникальных значений
candidate_targets = [c for c in df_cls_sample.columns if any(k in c.lower() for k in ["satisf", "sentiment", "label", "target", "class", "rating"])]
if candidate_targets:
    target_cls = candidate_targets[0]
else:
    nunique = df_cls_sample.nunique(dropna=True).sort_values()
    target_cls = next(c for c in nunique.index if nunique[c] <= 20 and "id" not in c.lower())

target_cls


'channel_name'

In [89]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier

# Разделяем признаки и целевую переменную
X_cls = df_cls_sample.drop(columns=[target_cls])
y_cls = df_cls_sample[target_cls]


In [90]:
# Делим на train/test; стратификация помогает сохранить пропорции классов
X_train, X_test, y_train, y_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=RANDOM_STATE, stratify=y_cls if y_cls.nunique() > 1 else None
)


In [91]:
# Для бустинга используем one-hot кодирование и заполнение пропусков
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

num_pipe = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(transformers=[
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])


In [92]:
# Бейзлайн-модель: GradientBoostingClassifier
clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", GradientBoostingClassifier(random_state=RANDOM_STATE))
])

clf.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [93]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Оценка качества классификации на тестовой выборке
y_pred = clf.predict(X_test)
metrics_cls = {
    "accuracy": accuracy_score(y_test, y_pred),
    "f1_macro": f1_score(y_test, y_pred, average="macro")
}
metrics_cls


{'accuracy': 0.7983706720977597, 'f1_macro': 0.32397360969413097}

In [94]:
# Отчёт по классам нужен для детальной оценки precision/recall/F1
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       Email       1.00      0.01      0.02       119
     Inbound       0.80      1.00      0.89      2729
     Outcall       0.68      0.04      0.07       589

    accuracy                           0.80      3437
   macro avg       0.83      0.35      0.32      3437
weighted avg       0.79      0.80      0.72      3437



## 3) Регрессия: cwurData.csv — бейзлайн на GradientBoostingRegressor


In [95]:
df_reg = pd.read_csv("cwurData.csv")

# Загрузили датасет регрессии; далее выберем целевой столбец
df_reg.shape


(2200, 14)

In [96]:
# Выбор target: приоритет world_rank, иначе - подходящий числовой столбец
preferred_targets = ["world_rank", "score", "rank"]
target_reg = next((t for t in preferred_targets if t in df_reg.columns), None)

if target_reg is None:
    num_candidates = df_reg.select_dtypes(include=[np.number]).columns.tolist()
    target_reg = num_candidates[0] if num_candidates else df_reg.columns[-1]

target_reg


'world_rank'

In [97]:
from sklearn.ensemble import GradientBoostingRegressor

# Разделяем признаки и таргет, затем train/test split
X_reg = df_reg.drop(columns=[target_reg])
y_reg = df_reg[target_reg]

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)


In [98]:
# Препроцессинг: заполнение пропусков и one-hot кодирование категориальных признаков
num_cols_r = Xr_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_r = [c for c in Xr_train.columns if c not in num_cols_r]

num_pipe_r = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
cat_pipe_r = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocess_r = ColumnTransformer(transformers=[
    ("num", num_pipe_r, num_cols_r),
    ("cat", cat_pipe_r, cat_cols_r)
])


In [99]:
# Бейзлайн-модель: GradientBoostingRegressor
reg = Pipeline(steps=[
    ("preprocess", preprocess_r),
    ("model", GradientBoostingRegressor(random_state=RANDOM_STATE))
])

reg.fit(Xr_train, yr_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [100]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Оценка качества регрессии на тестовой выборке
yr_pred = reg.predict(Xr_test)
metrics_reg = {
    "MAE": mean_absolute_error(yr_test, yr_pred),
    "RMSE": mean_squared_error(yr_test, yr_pred) ** 0.5,
    "R2": r2_score(yr_test, yr_pred)
}
metrics_reg


{'MAE': 11.277082472926155,
 'RMSE': 16.03298645835907,
 'R2': 0.9971960859224167}

## 4) Итоговая сводка метрик (бейзлайн)
Сведём результаты в таблицы для отчёта.


In [101]:
# Таблица метрик для классификации
summary_cls = pd.DataFrame([metrics_cls])
summary_cls


,accuracy,f1_macro
0,0.798371,0.323974


In [102]:
# Таблица метрик для регрессии
summary_reg = pd.DataFrame([metrics_reg])
summary_reg


,MAE,RMSE,R2
0,11.277082,16.032986,0.997196


## 3) Улучшение бейзлайна


### 3.1 Формулировка гипотез

#### Классификация (GradientBoostingClassifier)
1. Подбор числа деревьев (`n_estimators`) улучшит качество за счёт более точной аппроксимации.
2. Настройка скорости обучения (`learning_rate`) позволит избежать переобучения.
3. Ограничение глубины базовых деревьев (`max_depth`) повысит обобщающую способность модели.

#### Регрессия (GradientBoostingRegressor)
1. Подбор `n_estimators` и `learning_rate` снизит ошибку RMSE.
2. Ограничение глубины деревьев уменьшит переобучение.
3. Использование кросс-валидации позволит выбрать более устойчивую модель.


### 3.2 Проверка гипотез: классификация
Подберём гиперпараметры градиентного бустинга с помощью GridSearchCV.


In [103]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# Улучшенная модель градиентного бустинга для классификации
clf_improved = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", GradientBoostingClassifier(random_state=RANDOM_STATE))
])


In [104]:
# Сетка гиперпараметров для классификации
param_grid_cls = {
    "model__n_estimators": [100, 200],
    "model__learning_rate": [0.05, 0.1],
    "model__max_depth": [3, 5]
}

# Используем F1-macro как основную метрику кросс-валидации
grid_cls = GridSearchCV(
    clf_improved,
    param_grid=param_grid_cls,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1
)


In [105]:
grid_cls.fit(X_train, y_train)

# Сохраняем лучшую модель классификации
best_clf = grid_cls.best_estimator_
grid_cls.best_params_


{'model__learning_rate': 0.1,
 'model__max_depth': 5,
 'model__n_estimators': 200}

In [106]:
from sklearn.metrics import accuracy_score, f1_score

# Оцениваем улучшенную модель классификации на тестовой выборке
y_pred_imp = best_clf.predict(X_test)
metrics_cls_improved = {
    "accuracy": accuracy_score(y_test, y_pred_imp),
    "f1_macro": f1_score(y_test, y_pred_imp, average="macro")
}
metrics_cls_improved


{'accuracy': 0.7998254291533314, 'f1_macro': 0.34049076568918024}

### 3.3 Проверка гипотез: регрессия
Подберём гиперпараметры градиентного бустинга для задачи регрессии.


In [107]:
from sklearn.ensemble import GradientBoostingRegressor

# Улучшенная модель градиентного бустинга для регрессии
reg_improved = Pipeline(steps=[
    ("preprocess", preprocess_r),
    ("model", GradientBoostingRegressor(random_state=RANDOM_STATE))
])


In [108]:
# Сетка гиперпараметров для регрессии
param_grid_reg = {
    "model__n_estimators": [100, 200],
    "model__learning_rate": [0.05, 0.1],
    "model__max_depth": [3, 5]
}

# Используем отрицательный RMSE как целевую метрику
grid_reg = GridSearchCV(
    reg_improved,
    param_grid=param_grid_reg,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)


In [109]:
grid_reg.fit(Xr_train, yr_train)

# Сохраняем лучшую модель регрессии
best_reg = grid_reg.best_estimator_
grid_reg.best_params_


{'model__learning_rate': 0.05,
 'model__max_depth': 5,
 'model__n_estimators': 200}

In [110]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Оцениваем улучшенную модель регрессии на тестовой выборке
yr_pred_imp = best_reg.predict(Xr_test)
metrics_reg_improved = {
    "MAE": mean_absolute_error(yr_test, yr_pred_imp),
    "RMSE": mean_squared_error(yr_test, yr_pred_imp) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_imp)
}
metrics_reg_improved


{'MAE': 6.122733071472967,
 'RMSE': 10.984879397399505,
 'R2': 0.9986837862120297}

### 3.4 Сравнение с бейзлайном (пункт 2)


In [111]:
# Сравнение классификации: бейзлайн vs улучшенная модель
compare_cls = pd.DataFrame(
    [metrics_cls, metrics_cls_improved],
    index=["Baseline", "Improved"]
)
compare_cls


,accuracy,f1_macro
Baseline,0.798371,0.323974
Improved,0.799825,0.340491


In [112]:
# Сравнение регрессии: бейзлайн vs улучшенная модель
compare_reg = pd.DataFrame(
    [metrics_reg, metrics_reg_improved],
    index=["Baseline", "Improved"]
)
compare_reg


,MAE,RMSE,R2
Baseline,11.277082,16.032986,0.997196
Improved,6.122733,10.984879,0.998684


### 3.5 Выводы

1. В задаче классификации улучшенный бейзлайн на основе градиентного бустинга показал небольшое, но стабильное улучшение качества по сравнению с базовой моделью. Значение `accuracy` немного возросло, а значение `f1_macro` увеличилось более заметно, что свидетельствует о более сбалансированном качестве классификации по всем классам.

2. В задаче регрессии улучшенный бейзлайн продемонстрировал существенное улучшение качества по сравнению с базовой моделью. Значения MAE и RMSE значительно снизились, при этом значение R² увеличилось, что указывает на более точную аппроксимацию целевой переменной и снижение средней ошибки предсказаний.

3. В целом, результаты подтверждают выдвинутые гипотезы: подбор гиперпараметров и использование кросс-валидации позволили существенно повысить качество моделей градиентного бустинга по сравнению с бейзлайнами из пункта 2.


## 4) Имплементация алгоритма машинного обучения (Gradient Boosting)

В этом разделе:
- реализуем собственный градиентный бустинг для классификации и регрессии;
- обучим модели на тех же данных;
- оценим качество по выбранным метрикам;
- сравним с результатами из пункта 2 (sklearn бейзлайн);
- добавим техники из пункта 3с (подобранные гиперпараметры) и сравним с пунктом 3.


In [113]:
import numpy as np

# Фиксируем генератор случайных чисел для воспроизводимости
RNG = np.random.default_rng(42)


### 4.1 Подготовка данных для собственной реализации (one-hot + пропуски)
Собственная реализация работает с числовой матрицей, поэтому используем `get_dummies` и заполняем пропуски.


In [114]:
# Подготовка признаков для классификации: пропуски + one-hot + выравнивание колонок
X_train_p = X_train.copy()
X_test_p = X_test.copy()

for c in X_train_p.columns:
    if pd.api.types.is_numeric_dtype(X_train_p[c]):
        med = X_train_p[c].median()
        X_train_p[c] = X_train_p[c].fillna(med)
        X_test_p[c] = X_test_p[c].fillna(med)
    else:
        mode = X_train_p[c].mode(dropna=True)
        fill_val = mode.iloc[0] if len(mode) else "missing"
        X_train_p[c] = X_train_p[c].fillna(fill_val)
        X_test_p[c] = X_test_p[c].fillna(fill_val)

X_train_ohe = pd.get_dummies(X_train_p, drop_first=False)
X_test_ohe = pd.get_dummies(X_test_p, drop_first=False)
X_train_ohe, X_test_ohe = X_train_ohe.align(X_test_ohe, join="left", axis=1, fill_value=0)

X_train_np = X_train_ohe.to_numpy()
X_test_np = X_test_ohe.to_numpy()


In [115]:
# Подготовка признаков для регрессии: пропуски + one-hot + выравнивание колонок
Xr_train_p = Xr_train.copy()
Xr_test_p = Xr_test.copy()

for c in Xr_train_p.columns:
    if pd.api.types.is_numeric_dtype(Xr_train_p[c]):
        med = Xr_train_p[c].median()
        Xr_train_p[c] = Xr_train_p[c].fillna(med)
        Xr_test_p[c] = Xr_test_p[c].fillna(med)
    else:
        mode = Xr_train_p[c].mode(dropna=True)
        fill_val = mode.iloc[0] if len(mode) else "missing"
        Xr_train_p[c] = Xr_train_p[c].fillna(fill_val)
        Xr_test_p[c] = Xr_test_p[c].fillna(fill_val)

Xr_train_ohe = pd.get_dummies(Xr_train_p, drop_first=False)
Xr_test_ohe = pd.get_dummies(Xr_test_p, drop_first=False)
Xr_train_ohe, Xr_test_ohe = Xr_train_ohe.align(Xr_test_ohe, join="left", axis=1, fill_value=0)

Xr_train_np = Xr_train_ohe.to_numpy()
Xr_test_np = Xr_test_ohe.to_numpy()


### 4.2 Имплементация градиентного бустинга

Идея:
- для регрессии: бустинг по MSE (предсказываем остатки);
- для классификации: упрощённая версия через бустинг вероятностей (логистическая функция) и обучение на псевдо-остатках.

Базовый алгоритм реализуем через ансамбль `DecisionTreeRegressor` как слабых моделей.


In [116]:
from sklearn.tree import DecisionTreeRegressor

# Упрощённый Gradient Boosting для регрессии (MSE)
class MyGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.init_ = None
        self.models_ = []

    def fit(self, X, y):
        # Инициализируем предсказание средним значением
        rng = np.random.default_rng(self.random_state)
        self.init_ = float(np.mean(y))
        pred = np.full(len(y), self.init_, dtype=float)

        self.models_ = []
        for _ in range(self.n_estimators):
            resid = y - pred
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=int(rng.integers(0, 1_000_000)))
            tree.fit(X, resid)

            pred += self.learning_rate * tree.predict(X)
            self.models_.append(tree)
        return self

    def predict(self, X):
        # Суммируем вклад всех слабых моделей
        pred = np.full(X.shape[0], self.init_, dtype=float)
        for tree in self.models_:
            pred += self.learning_rate * tree.predict(X)
        return pred


In [117]:
# Упрощённый Gradient Boosting для бинарной классификации (логистическая функция)
class MyGradientBoostingBinaryClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.init_ = None
        self.models_ = []

    @staticmethod
    def _sigmoid(z):
        # Сигмоида для вероятностей
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # Обучаем на псевдо-остатках для логистической потери
        rng = np.random.default_rng(self.random_state)
        y = y.astype(int)

        p = np.clip(np.mean(y), 1e-6, 1 - 1e-6)
        self.init_ = float(np.log(p / (1 - p)))
        F = np.full(len(y), self.init_, dtype=float)

        self.models_ = []
        for _ in range(self.n_estimators):
            prob = self._sigmoid(F)
            resid = y - prob
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=int(rng.integers(0, 1_000_000)))
            tree.fit(X, resid)

            F += self.learning_rate * tree.predict(X)
            self.models_.append(tree)
        return self

    def predict_proba(self, X):
        # Возвращаем вероятности классов
        F = np.full(X.shape[0], self.init_, dtype=float)
        for tree in self.models_:
            F += self.learning_rate * tree.predict(X)
        p1 = self._sigmoid(F)
        return np.vstack([1 - p1, p1]).T

    def predict(self, X):
        # Классы получаем по порогу 0.5
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)


### 4.3 Обучение и оценка собственных моделей (сравнение с пунктом 2)

Важно: собственная классификация реализована для **бинарного** случая.  
Если классов больше двух — используем стратегию One-vs-Rest (поочерёдно обучаем бинарные модели).


In [118]:
from sklearn.preprocessing import LabelEncoder

# Кодируем классы в числа, чтобы работать с собственной реализацией
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)
n_classes = len(le.classes_)

n_classes


3

In [119]:
# Реализация One-vs-Rest для многоклассовой классификации
class MyGradientBoostingOVR:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.models_ = []

    def fit(self, X, y, n_classes):
        # Обучаем по одной бинарной модели на каждый класс
        self.models_ = []
        for k in range(n_classes):
            y_bin = (y == k).astype(int)
            m = MyGradientBoostingBinaryClassifier(
                n_estimators=self.n_estimators,
                learning_rate=self.learning_rate,
                max_depth=self.max_depth,
                random_state=self.random_state + k
            )
            m.fit(X, y_bin)
            self.models_.append(m)
        return self

    def predict(self, X):
        # Выбираем класс с максимальной вероятностью "1" среди OVR моделей
        probs = np.vstack([m.predict_proba(X)[:, 1] for m in self.models_]).T
        return np.argmax(probs, axis=1)


In [120]:
from sklearn.metrics import accuracy_score, f1_score

# Собственный бустинг для классификации (бейзлайн-версия)
my_gb_cls = MyGradientBoostingOVR(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=RANDOM_STATE)
my_gb_cls.fit(X_train_np, y_train_enc, n_classes)

y_pred_my_enc = my_gb_cls.predict(X_test_np)
y_pred_my = le.inverse_transform(y_pred_my_enc)

metrics_my_cls = {
    "accuracy": accuracy_score(y_test, y_pred_my),
    "f1_macro": f1_score(y_test, y_pred_my, average="macro")
}
metrics_my_cls


{'accuracy': 0.7940064009310445, 'f1_macro': 0.29505892528922045}

In [121]:
# Собственный бустинг для регрессии (бейзлайн-версия)
my_gb_reg = MyGradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=RANDOM_STATE)
my_gb_reg.fit(Xr_train_np, yr_train.to_numpy())

yr_pred_my = my_gb_reg.predict(Xr_test_np)
metrics_my_reg = {
    "MAE": mean_absolute_error(yr_test, yr_pred_my),
    "RMSE": mean_squared_error(yr_test, yr_pred_my) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_my)
}
metrics_my_reg


{'MAE': 11.26244999285648, 'RMSE': 16.01520145400263, 'R2': 0.9972023031004501}

In [122]:
# Сравнение: sklearn бейзлайн (п.2) vs собственная реализация (п.4)
compare_impl_cls_vs_p2 = pd.DataFrame(
    [metrics_cls, metrics_my_cls],
    index=["Sklearn Baseline (p2)", "My GB (p4)"]
)
compare_impl_reg_vs_p2 = pd.DataFrame(
    [metrics_reg, metrics_my_reg],
    index=["Sklearn Baseline (p2)", "My GB (p4)"]
)

compare_impl_cls_vs_p2, compare_impl_reg_vs_p2


(                       accuracy  f1_macro
 Sklearn Baseline (p2)  0.798371  0.323974
 My GB (p4)             0.794006  0.295059,
                              MAE       RMSE        R2
 Sklearn Baseline (p2)  11.277082  16.032986  0.997196
 My GB (p4)             11.262450  16.015201  0.997202)

### 4.4 Выводы (сравнение с пунктом 2)

1. Собственная реализация градиентного бустинга корректно воспроизводит принцип последовательного обучения слабых моделей на ошибках предыдущих.
2. Различия с sklearn объясняются упрощениями: отсутствием оптимизаций, регуляризации, подбора порогов и специализированной реализации логистической потери.
3. При этом метрики позволяют сравнить качество и убедиться, что реализация работоспособна на исходных данных.


### 4.5 Добавление техник из улучшенного бейзлайна (пункт 3с) и повторное обучение
Используем лучшие параметры из `grid_cls.best_params_` и `grid_reg.best_params_`.


In [123]:
# Берём лучшие параметры из пункта 3
best_params_cls = grid_cls.best_params_
best_params_reg = grid_reg.best_params_

best_params_cls, best_params_reg


({'model__learning_rate': 0.1,
  'model__max_depth': 5,
  'model__n_estimators': 200},
 {'model__learning_rate': 0.05,
  'model__max_depth': 5,
  'model__n_estimators': 200})

In [124]:
# Улучшенная собственная классификация с параметрами из п.3с
my_gb_cls_imp = MyGradientBoostingOVR(
    n_estimators=best_params_cls.get("model__n_estimators", 100),
    learning_rate=best_params_cls.get("model__learning_rate", 0.1),
    max_depth=best_params_cls.get("model__max_depth", 3),
    random_state=RANDOM_STATE
)
my_gb_cls_imp.fit(X_train_np, y_train_enc, n_classes)

y_pred_my_imp_enc = my_gb_cls_imp.predict(X_test_np)
y_pred_my_imp = le.inverse_transform(y_pred_my_imp_enc)

metrics_my_cls_improved = {
    "accuracy": accuracy_score(y_test, y_pred_my_imp),
    "f1_macro": f1_score(y_test, y_pred_my_imp, average="macro")
}
metrics_my_cls_improved


{'accuracy': 0.7992435263311027, 'f1_macro': 0.3207377027630991}

In [125]:
# Улучшенная собственная регрессия с параметрами из п.3с
my_gb_reg_imp = MyGradientBoostingRegressor(
    n_estimators=best_params_reg.get("model__n_estimators", 100),
    learning_rate=best_params_reg.get("model__learning_rate", 0.1),
    max_depth=best_params_reg.get("model__max_depth", 3),
    random_state=RANDOM_STATE
)
my_gb_reg_imp.fit(Xr_train_np, yr_train.to_numpy())

yr_pred_my_imp = my_gb_reg_imp.predict(Xr_test_np)
metrics_my_reg_improved = {
    "MAE": mean_absolute_error(yr_test, yr_pred_my_imp),
    "RMSE": mean_squared_error(yr_test, yr_pred_my_imp) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_my_imp)
}
metrics_my_reg_improved


{'MAE': 6.110842756149755,
 'RMSE': 11.037142763217341,
 'R2': 0.9986712319722202}

In [126]:
# Сравнение: sklearn improved (п.3) vs моя improved (п.4)
compare_impl_cls_vs_p3 = pd.DataFrame(
    [metrics_cls_improved, metrics_my_cls_improved],
    index=["Sklearn Improved (p3)", "My Improved (p4)"]
)
compare_impl_reg_vs_p3 = pd.DataFrame(
    [metrics_reg_improved, metrics_my_reg_improved],
    index=["Sklearn Improved (p3)", "My Improved (p4)"]
)

compare_impl_cls_vs_p3, compare_impl_reg_vs_p3


(                       accuracy  f1_macro
 Sklearn Improved (p3)  0.799825  0.340491
 My Improved (p4)       0.799244  0.320738,
                             MAE       RMSE        R2
 Sklearn Improved (p3)  6.122733  10.984879  0.998684
 My Improved (p4)       6.110843  11.037143  0.998671)

### 4.6 Выводы (сравнение с пунктом 3)

1. Добавление техник из пункта 3с (подобранные гиперпараметры) позволяет приблизить качество собственной реализации к улучшенному бейзлайну sklearn.
2. Оставшиеся различия в метриках объясняются тем, что sklearn использует более точную оптимизацию и регуляризацию, а собственная реализация является упрощённой.
3. В целом, улучшенная собственная реализация демонстрирует корректную работу алгоритма градиентного бустинга и сопоставимое качество после настройки параметров.
